In [191]:
### load required packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# Low dimensional data

In [192]:
### read data
lowDim_data = pd.read_csv('../data/lowDim_dataset.csv')
lowDim_data

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
0,19.678858,0,1.59,0.00,0.0,0.00,0.24,1.35,0.73,2.58,...,0.12,0.00,4.55,0.00,1.72,0.00,0.49,0.98,0.00,1.309683
1,17.842989,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.62,...,0.27,0.00,4.87,0.00,0.81,0.27,0.27,0.00,0.00,1.719547
2,22.108788,1,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,2.12,0.00,0.00,0.00,2.12,0.996210
3,15.355899,0,0.00,0.00,0.0,0.56,0.00,0.00,0.00,0.00,...,0.00,0.00,1.12,0.00,0.00,0.00,0.00,0.00,0.00,1.504077
4,16.787813,1,1.81,0.00,0.0,0.00,0.00,0.00,0.00,1.81,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.327864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,23.456961,1,0.00,0.12,0.0,0.00,0.00,0.12,0.00,0.19,...,0.12,0.00,1.74,0.06,1.23,0.00,0.25,0.06,2.26,0.705076
471,18.449855,1,0.77,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.55,0.00,0.77,0.00,0.00,0.77,0.00,1.150572
472,18.646830,1,0.91,0.18,0.0,0.00,0.00,0.00,0.00,3.66,...,0.82,0.00,0.82,0.00,0.45,0.00,0.00,0.00,1.37,0.233490
473,25.131092,0,0.00,0.36,0.0,0.36,0.72,1.08,0.36,0.72,...,0.36,0.36,1.08,0.00,2.53,0.00,0.00,0.00,0.00,2.912351


## 1. Propensity score estimation
## 1.1 Select variables to estimate propensity score

Note: Select criteria is t-statistics. We keep the covariate with has t-statistics larger than t_prop=2. 

In [193]:
### perfom K logistic regression to select for V (covariates for propensity score estimation)

K = lowDim_data.shape[1]-2
t_prop = 2 # threshold to select covariate
T = lowDim_data[['A']]

t_stat_prop = np.array(())
for i in range(1, K+1):
    idx = 'V'+str(i)
    X = lowDim_data[[idx]]

    log_reg = sm.Logit(T, X).fit() 
    cov = log_reg.cov_params()
    std_err = np.sqrt(np.diag(cov))
    t_stat_prop = np.append(t_stat_prop,np.array(round(log_reg.params / std_err,2)))

Optimization terminated successfully.
         Current function value: 0.684936
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686798
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.692697
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.677862
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.687807
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.689411
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691128
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.665423
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.680357
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.691334
  

## 1.2 Estimate propensity score with L2 logistic

In [194]:
### propensity score estimation with logistic regression with L2 penalty
# V is a subset of covariates (selected by t_stat_prop)
V = lowDim_data.iloc[:,2:].iloc[:,abs(t_stat_prop)>=t_prop]
T = np.array(T).ravel()

clf = LogisticRegression(random_state=0, solver='lbfgs', penalty='l2', C=1.0).fit(V, T)
PS = clf.predict_proba(V)[:,1]

## 1.3 Calculate weights according to PS

In [195]:
### calculate weights
weights = T/PS + (1-T)/(1-PS)

## 2. ATE estimation
## 2.1 Select variables to estimate ATE

Note: Select criteria is t-statistics. We keep the covariate with has t-statistics larger than t_reg=2. 

In [196]:
### perform K linear regressions to select for Z
t_reg = 2 # threshold to select covariate
Y = lowDim_data[['Y']]

t_stat_reg = np.array(())
for i in range(1, K+1):
    idx = 'V'+str(i)
    X = lowDim_data[['A',idx]]
    X = sm.add_constant(X) # adding a constant
    
    linear_reg = sm.OLS(Y, X).fit() 
    cov = linear_reg.cov_params()
    std_err = np.sqrt(np.diag(cov))
    t_stat_reg = np.append(t_stat_reg,np.array(round(linear_reg.params[2] / std_err[2],2)))

## 2.2 Estimate ATE with weighted linear regression 

$Y_i = \alpha_0+\tau T_i+\alpha_{11}Z_{i1}+\alpha_{12}Z_{i2}+ ... +\alpha_{1m}Z_{im}+\alpha_{21}(Z_{i1}-\bar{Z_1})T_i+\alpha_{22}(Z_{i2}-\bar{Z_2})T_i+...+\alpha_{2m}(Z_{im}-\bar{Z_m})T_i+\epsilon_i$

In [197]:
### weighted least square linear regression on selected covariates Z

# construct required independent variables

T = lowDim_data['A']

Z = lowDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg]
centralized_Z = lowDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg] - \
                lowDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg].mean(axis=0)
centralized_Z_T = centralized_Z.multiply(T, axis='index')

X_train = pd.concat([T,Z,centralized_Z,centralized_Z_T],axis=1)
Y_train = np.array(Y).ravel()

In [198]:
reg = LinearRegression(fit_intercept=True).fit(X_train, Y_train, sample_weight=weights)
ATE = reg.coef_[0]
print("Estimated ATE for low-dim data is {:.2f}".format(ATE))

Estimated ATE for low-dim data is 2.51


# Repeat for high dimensional data

In [199]:
### read data
highDim_data = pd.read_csv('../data/highDim_dataset.csv')
highDim_data

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185
0,-11.682472,1,0,1,2,16,3,-1,13,-0.13,...,5,7,8,6,8,-1,-1,-1,-1,-1
1,-13.176546,0,1,1,12,14,14,14,13,0.24,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-2.195401,1,0,1,21,22,10,10,14,0.27,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-0.005454,1,1,1,9,20,11,2,10,0.09,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
4,-1.987538,1,1,1,7,16,16,11,6,0.15,...,70,70,80,70,80,-10,-10,-10,-10,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-15.054763,0,0,1,14,18,19,19,10,0.36,...,7,8,9,10,9,6,8,8,10,8
1996,-8.310797,0,0,1,21,22,15,15,12,0.02,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1997,-15.645703,0,1,1,14,20,5,5,3,0.56,...,6,6,8,7,5,6,5,7,7,6
1998,-18.302366,1,0,1,2,16,7,-1,11,0.50,...,7,8,8,7,6,-1,-1,-1,-1,-1


In [200]:
### perfom K logistic regression to select for V (covariates for propensity score estimation)

K = highDim_data.shape[1]-2
t_prop = 2 # threshold to select covariate
T = highDim_data[['A']]

t_stat_prop = np.array(())
for i in range(1, K+1):
    idx = 'V'+str(i)
    X = highDim_data[[idx]]

    log_reg = sm.Logit(T, X).fit() 
    cov = log_reg.cov_params()
    std_err = np.sqrt(np.diag(cov))
    t_stat_prop = np.append(t_stat_prop,np.array(round(log_reg.params / std_err,2)))

Optimization terminated successfully.
         Current function value: 0.688663
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689171
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688522
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.688663
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690168
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690028
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689965
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.691124
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691320
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686929
  

In [201]:
### propensity score estimation with logistic regression with L2 penalty
# V is a subset of covariates (selected by t_stat_prop)
V = highDim_data.iloc[:,2:].iloc[:,abs(t_stat_prop)>=t_prop]
T = np.array(T).ravel()

clf = LogisticRegression(random_state=0, solver='newton-cg', penalty='l2', C=1.0).fit(V, T)
PS = clf.predict_proba(V)[:,1]

In [202]:
### calculate weights
weights = T/PS + (1-T)/(1-PS)

In [203]:
### perform K linear regressions to select for Z
t_reg = 2 # threshold to select covariate
Y = highDim_data[['Y']]

t_stat_reg = np.array(())
for i in range(1, K+1):
    idx = 'V'+str(i)
    X = highDim_data[['A',idx]]
    X = sm.add_constant(X) # adding a constant
    
    linear_reg = sm.OLS(Y, X).fit() 
    cov = linear_reg.cov_params()
    std_err = np.sqrt(np.diag(cov))
    t_stat_reg = np.append(t_stat_reg,np.array(round(linear_reg.params[2] / std_err[2],2)))

In [204]:
### weighted least square linear regression on selected covariates Z

# construct required independent variables

T = highDim_data['A']

Z = highDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg]
centralized_Z = highDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg] - \
                highDim_data.iloc[:,2:].iloc[:,abs(t_stat_reg)>t_reg].mean(axis=0)
centralized_Z_T = centralized_Z.multiply(T, axis='index')

X_train = pd.concat([T,Z,centralized_Z,centralized_Z_T],axis=1)
Y_train = np.array(Y).ravel()

In [205]:
reg = LinearRegression(fit_intercept=True).fit(X_train, Y_train, sample_weight=weights)
ATE = reg.coef_[0]
print("Estimated ATE for high-dim data is {:.2f}".format(ATE))

Estimated ATE for high-dim data is -2.97
